In [1]:
import os
import sys
import pprint

base_dir = 'C:\Documents\Thesis_ssd\MasterThesis-2.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.BaselineHelperFunctions import BaselineHelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.Models import Models
from Classes.Modeling.RandomGridSearch import RandomGridSearch
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
import json

helper = BaselineHelperFunctions()
%load_ext tensorboard

In [2]:
num_classes = 2
loadData = LoadData(num_classes = 2, isBalanced = True)
shuffle = False
full_ds, train_ds, val_ds, test_ds = loadData.getDatasets(shuffle = shuffle)
handler = DataHandler()
pp = pprint.PrettyPrinter(indent=4)
helper = BaselineHelperFunctions()

In [3]:
hyper_grid = {
        "batch_size" : [8, 16, 32, 64, 128, 256],
        "epochs" : [30, 35, 40],
        "learning_rate" : [0.1, 0.01, 0.001, 0.0001, 0.00001],
        "optimizer" : ["adam", "rmsprop", "sgd"]
    }
model_grid = {
    "start_neurons" : [4,8,16, 32, 64, 128, 256, 512],
    "dropout_rate" : [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0],
    "filters" : [11, 13, 15, 17, 19, 21, 23, 25],
    "kernel_size" : [3, 5, 7, 9, 11, 13],
    "padding" : ["same"],
    "l2_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
    "l1_r" : [0.3, 0.2, 0.1, 0.01, 0.001, 0.0001],
    "activation" : ["relu", "sigmoid", "softmax", "tanh"],
    "output_layer_activation" : ["softmax", "sigmoid"]
}


model_nr = 4
test_mode = False
use_scaler = True
use_noise_augmentor = True
detrend = False
use_minmax = False
n_picks = 40
use_tensorboard = False

randomGridSearch = RandomGridSearch(train_ds, val_ds, test_ds, model_nr, test_mode, detrend,
                                    use_scaler, use_noise_augmentor, use_minmax, n_picks, 
                                    hyper_grid = hyper_grid, model_grid = model_grid, 
                                    num_classes = num_classes, use_tensorboard = use_tensorboard)

In [4]:
def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))
if use_tensorboard:
    clear_tensorboard_dir()
    %tensorboard --logdir tensorboard_dir/fit

In [ ]:
results, highest_test_accuracy_index, highest_train_accuracy_index, highest_test_precision_index, highest_test_recall_index= randomGridSearch.fit()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (32, 128)                 3138560   
_________________________________________________________________
dropout (Dropout)            (32, 128)                 0         
_________________________________________________________________
batch_normalization (BatchNo (32, 128)                 512       
_________________________________________________________________
dense (Dense)                (32, 2)                   258       
Total params: 3,139,330
Trainable params: 3,139,074
Non-trainable params: 256
_________________________________________________________________
Starting: 
{'batch_size': 32, 'epochs': 35, 'learning_rate': 0.1, 'optimizer': 'sgd'}
---------------------------------------------------------------------------------
{   'activation': 'sigmoid',
    'dropout_rate': 0.1,
    'filters': 25,
 

In [ ]:
dictionaries = randomGridSearch.read_results()
print(dictionaries)
use_tensorboard = True

**Highest test accuracy**

In [ ]:
model_test_accuracy = randomGridSearch.fit_from_result(dictionaries, highest_test_accuracy_index, use_tensorboard = use_tensorboard)

**Highest train accuracy**

In [ ]:
model_train_accuracy = randomGridSearch.fit_from_result(dictionaries, highest_train_accuracy_index, use_tensorboard = use_tensorboard)

**Highest precision**

In [ ]:
model_test_precision = randomGridSearch.fit_from_result(dictionaries, highest_test_precision_index, use_tensorboard = use_tensorboard)

**Highest recall**

In [ ]:
model_test_recall = randomGridSearch.fit_from_result(dictionaries, highest_test_recall_index, use_tensorboard = use_tensorboard)